In [1]:
from prepare_language_folder import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

In [2]:
#data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'
data_collection_name = 'MultiplEYE_ZH_CH_Zurich_1_2025'


If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [3]:
prepare_language_folder(data_collection_name)

No zipped data found for 079_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 023_ZH_CH_1_ET1. Proceeding with existing data.
No zipped data found for 084_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 009_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 053_ZH_CH_1_ET1. Proceeding with existing data.
No zipped data found for 057_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 080_ZH_CH_1_ET1. Proceeding with existing data.
No zipped data found for 027_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 103_ZH_CH_1_ET1. Proceeding with existing data.
No zipped data found for 061_ZH_CH_1_ET1. Proceeding with existing data.
No zipped data found for 011_ZH_CH_1_ET1. Proceeding with existing data.
No zipped data found for 015_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 098_ZH_CH_1_ET2. Proceeding with existing data.
No zipped data found for 065_ZH_CH_1_ET2. Proceedin

In [4]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)
multipleye_sq

Folder test_sessions does not match the regex pattern \d\d\d_ZH_CH_1_ET\d. Not considered as session.
Folder __MACOSX does not match the regex pattern \d\d\d_ZH_CH_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_ZH_CH_1_ET\d. Not considered as session.


Converting EDF to ASC: 100%|██████████| 212/212 [00:00<00:00, 28917.77it/s]


ValueError: All arrays must be of the same length

In [ ]:
multipleye_sq.create_sanity_check_report(overwrite=False)

In [ ]:
multipleye_sq.preprocess_eye_tracking_data()

In [ ]:
sessions = [s for s in multipleye_sq]

In [ ]:
sessions[0]

Step 1 pf peyepline: create the gaze frame.

	-- data collection folder
	---- ...
	---- fixations
	---- saccades(?)
	---- reading_measures
	---- raw_data (i.e. gaze sample csv)